## Setup

In [2]:
# Import TensorFlow
import tensorflow as tf
import numpy as np
import os
import time

## Download Dataset Shakespeare

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


## Load Data

In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8') # Buka file
print(f'Length of text: {len(text)} characters') # cetak panjang teks

Length of text: 1115394 characters


In [5]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# The unique characters in the file
vocab = sorted(set(text)) # buat kosakata
print(f'{len(vocab)} unique characters') # cetak unique characters

65 unique characters


## Olah Teks

In [7]:
# Menentukan beberapa teks contoh sebagai list
example_texts = ['abcdefg', 'xyz']

# Membagi setiap teks menjadi karakter
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

# Menampilkan hasilnya
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [8]:
# untuk mengonversi karakter menjadi indeks numerik.
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [9]:
# Menggunakan layer StringLookup sebelumnya untuk mengonversi karakter Unicode menjadi indeks numerik
# 'chars' adalah variabel yang berisi karakter Unicode
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [10]:
# Membuat layer StringLookup baru untuk mengonversi indeks numerik kembali menjadi karakter Unicode
# Menggunakan vocabulary dari ids_from_chars dan mengaktifkan invert=True
# mask_token=None menunjukkan bahwa tidak ada token mask yang digunakan
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [11]:
# Lapisan ini mengconvert kembali karakter dari vektor ID, dan mengembalikannya sebagai karakter tf.RaggedTensor
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
# Menggabungkan karakter Unicode dalam 'chars' menjadi string tunggal
# axis=-1 menunjukkan bahwa penggabungan dilakukan sepanjang dimensi terakhir
# .numpy() mengonversi hasil tensor TensorFlow menjadi nilai NumPy
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
    # Menggunakan layer chars_from_ids untuk mengonversi indeks numerik kembali menjadi karakter Unicode
    # Menggabungkan karakter Unicode menjadi string tunggal
    # axis=-1 menunjukkan bahwa penggabungan dilakukan sepanjang dimensi terakhir
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

## Prediksi

In [14]:
# Menggunakan layer ids_from_chars untuk mengonversi teks menjadi indeks numerik
# tf.strings.unicode_split(text, 'UTF-8') membagi teks menjadi karakter Unicode
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [15]:
# Membuat objek tf.data.Dataset dari tensor all_ids
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
# Iterasi melalui 10 elemen pertama dari dataset ids_dataset
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

# Menetapkan panjang urutan (sequence length) menjadi 100
seq_length = 100


F
i
r
s
t
 
C
i
t
i


In [17]:
# Metode batch memungkinkan Anda dengan mudah mengonversi karakter individual ini menjadi urutan ukuran yang diinginkan.
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [18]:
# Iterasi melalui 5 elemen pertama dari dataset sequences
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [19]:
# fungsi yang mengambil urutan sebagai masukan, menduplikasi, dan menggesernya untuk menyelaraskan masukan dan label untuk setiap langkah waktu
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

split_input_target(list("Tensorflow"))
dataset = sequences.map(split_input_target)

In [20]:
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [21]:
# Membuat Batch Training
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Buat Model

In [22]:
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [23]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)

        # Lapisan embedding untuk mengonversi indeks numerik menjadi vektor embedding
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

        # Lapisan GRU untuk memodelkan urutan data
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)

        # Lapisan dense untuk output dengan ukuran sejumlah vocab_size
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs

        # Menggunakan lapisan embedding
        x = self.embedding(x, training=training)

        # Jika states tidak diberikan, menginisialisasi states dengan nilai default dari lapisan GRU
        if states is None:
            states = self.gru.get_initial_state(x)

        # Memproses urutan data melalui lapisan GRU
        x, states = self.gru(x, initial_state=states, training=training)

        # Melalui lapisan dense untuk mendapatkan output
        x = self.dense(x, training=training)

        # Mengembalikan output dan states jika diperlukan
        if return_state:
            return x, states
        else:
            return x


In [24]:
# Membuat instance dari kelas MyModel
model = MyModel(
    vocab_size=vocab_size,       # Ukuran vokabular
    embedding_dim=embedding_dim, # Dimensi vektor embedding
    rnn_units=rnn_units           # Jumlah unit dalam lapisan GRU
)

## Uji Model

In [25]:
# Uji Model
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [26]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [28]:
sampled_indices

array([23, 34, 59, 59, 23, 62, 61, 22, 46, 60, 33, 47, 29, 18, 36, 38, 20,
       45, 35, 48, 10, 29, 62, 28, 10, 21, 64, 36, 59,  1, 53, 25, 48, 17,
       53, 34,  9, 24, 56, 43, 12, 44, 54, 46, 32, 58, 53, 29, 23, 21, 36,
       20, 13, 43,  4, 65, 43, 46, 45, 21, 64, 12, 63, 33, 51,  2, 22, 62,
       55, 59, 44, 56, 60, 17, 29, 22, 22, 46, 16, 59, 40, 11,  2,  8, 30,
        0,  2, 10,  3, 65, 57, 49, 10,  9,  4, 36,  4, 62,  4, 19])

In [29]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ank thee, gentle Percy; and be sure\nI count myself in nothing else so happy\nAs in a soul remembering'

Next Char Predictions:
 b'JUttJwvIguThPEWYGfVi3PwO3HyWt\nnLiDnU.Kqd;eogSsnPJHWG?d$zdgfHy;xTl IwptequDPIIgCta: -Q[UNK] 3!zrj3.$W$w$F'


## Train Model

In [30]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [31]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1906443, shape=(), dtype=float32)


In [32]:
tf.exp(example_batch_mean_loss).numpy()

66.065346

In [33]:
model.compile(optimizer='adam',loss=loss)

In [34]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [35]:
EPOCHS = 20

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 14s 55ms/step - loss: 2.7164
Epoch 2/20
172/172 [==============================] - 12s 55ms/step - loss: 1.9800
Epoch 3/20
172/172 [==============================] - 13s 56ms/step - loss: 1.7027
Epoch 4/20
172/172 [==============================] - 12s 58ms/step - loss: 1.5448
Epoch 5/20
172/172 [==============================] - 12s 58ms/step - loss: 1.4481
Epoch 6/20
172/172 [==============================] - 12s 59ms/step - loss: 1.3803
Epoch 7/20
172/172 [==============================] - 13s 58ms/step - loss: 1.3290
Epoch 8/20
172/172 [==============================] - 11s 57ms/step - loss: 1.2852
Epoch 9/20
172/172 [==============================] - 12s 58ms/step - loss: 1.2445
Epoch 10/20
172/172 [==============================] - 13s 58ms/step - loss: 1.2055
Epoch 11/20
172/172 [==============================] - 12s 59ms/step - loss: 1.1665
Epoch 12/20
172/172 [==============================] - 12s 60ms/step - loss: 1.1255
E

## Generate Teks

In [36]:
# Berikut ini membuat prediksi satu langkah
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [37]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [38]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Nay, I fear thy mind:
Let me enjurious time hath tomorrows to our free
Spies of so success? Henceforth free op nothing
They used with violent forest mother in a fight:
Your mistress is my lover: but he has it.
This your concluding face!
Dream of the dang-to-night should lift every one;
Wesher all virtuous and unsuee-foot, man, which are
As cheap as-little abreast, could with his fingers,
A thousand tongue to seek the base of time.
Hath crump consul? name her.

GRUMIO:
That in a brazen ir fire vengeance blood,
To wake it walking it to see.

ISABELLA:

us:
And thou shalt to Richard: what say you, undess
Of say she's upon the field: and you because
I am abrable. Good faith, he will go along
with thine eyes, flee from once again,
And he, that now 'Commanding that hath done me well:
Spore him some account! White-bear she breaks with said,
Heaven lost his eyes and eyebrows.

First Senator:
Then, in angels like toild,
Upon his name is as a brack,
Account thee by the first.

WARWICK:
Da

In [39]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nHenceful account thy name of life,\nI'll do't, some warrant for that place and time\nThe passonce of the key?\nThere spake me nothing: so much lengthen lengthens out\nof the form: then have strong night, as to the sun,\nThat doing thunders to me as we past the\npacing liber than his ownsil high, so by crooking is the\nwhite regain.\n\nSecond Lady:\nSound King Riscline of this truer Warwick?\nWe are not speak. Ah, what says God,\n'Tis bear thy soul's deign till thus hagled of this sweet;\nOr both passing well for thanks that doubt,\nBoth one my injunion frounds his hagrand\nbefore his word: see wot to what answer, you\n\nDUKE VINCENTIO:\nMore crimined Lord pretty boldnes, something steel, that\nsinging in the fight, time luirer power for in any\nbluel trut; for once to your fault was thought,\nTo smell out with a verge.\n\nFirst Servingman:\nI know thee, father, in faith, I know how here you shall find\nEnong up thus.'\n\nNurse:\nO Prothee, thou! and what news?\n\n

In [40]:
# Ekspor Model Generator
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Gramer, is King Henry leave?
By my trorn with old praying, may show
do purge Elsew silence in this 
